<img style="float: right; margin: 30px 15px 15px 15px;" src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTFzQj91sOlkeDFkg5HDbjtR4QJYmLXkfMNig&usqp=CAU" width="800" height="200" /> 
    
    
### <font color='navy'> Modelos no Lineales para Pronósticos. </font>

**Nombres:**
> `Cárdenas Gallardo Paula Daniela` | `733720` <br> `Haces López José Manuel` | `734759` <br> `Villa Domínguez Paulo Adrián` | `733773`

**Fecha:** Jueves 11 de Mayo de 2023
    
**Profesor:** Óscar David Jaramillo Zuluaga.
    
**Link Github**: [github.com](https://github.com/paucardenasg/Proyecto_MNLP)

# <font color='maroon'> Proyecto Final </font>

In [ ]:
# Librerías
import warnings
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from pmdarima.arima import auto_arima
from sklearn.naive_bayes import GaussianNB
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from sklearn.linear_model import LogisticRegression
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.preprocessing import PowerTransformer, MinMaxScaler, StandardScaler

# Funciones y Clases
from Utils_Multi import *
from Utils_Lineal import *
from Utils_Clasificacion import *
from Utils_Univariado import NN_maker

# Filtrando las advertencias
warnings.filterwarnings('ignore')

In [ ]:
# Carga de datos
data = pd.read_csv('./Data/dataset.csv')
# Eliminando la columna de index y Unit
data.drop(columns=['SN', 'Unit'], inplace=True)
# Poniendo el date a formato de fecha
data['Date'] = pd.to_datetime(data['Date'])
# Poniendo la fecha como str
data['Fecha_Str'] = data['Date'].dt.strftime('%Y-%m-%d')
# Cambiando el nombre de las columnas a español
data.rename(columns={'Date':'Fecha', 'Commodity':'Producto', 'Minimum':'Mínimo', 'Maximum':'Máximo', 'Average':'Promedio'}, inplace=True)

# --------------------------------------------------------------------------------------------------------------------

# Cargando el dataset del cambio de Rupia Nepali a Pesos
data_cambio = pd.read_excel('./Data/CambioMoneda.xlsx')
# Poniendo Fecha en formato de fech
data_cambio['Fecha'] = pd.to_datetime(data_cambio['Fecha'])
# Quitando la hora de la fecha
data_cambio['Fecha'] = data_cambio['Fecha'].dt.date
# Poniendo la fecha como str
data_cambio['Fecha_Str'] = data_cambio['Fecha'].astype(str)

# --------------------------------------------------------------------------------------------------------------------

# Obtener las clases de productos
productos_clase = data[['Producto']]
productos_clase = productos_clase.drop_duplicates()
# Tomar solo la primera palabra del valor de la columna de producto
productos_clase['Clase'] = productos_clase['Producto'].str.split(' ').str[0]
productos_clase['Clase'] = productos_clase['Clase'].str.split('(').str[0]

# Mergeando los datos con la clase de producto
data = data.merge(productos_clase, on='Producto')

# --------------------------------------------------------------------------------------------------------------------

# Juntando los dos datasets de tipo de cambio a pesos
data = data.merge(data_cambio.drop(columns=['Fecha']), on='Fecha_Str')

# --------------------------------------------------------------------------------------------------------------------

# Dataset con el precio del dolar
dolar = pd.read_excel('./Data/PrecioDolar.xlsx')
dolar['Fecha'] = pd.to_datetime(dolar['Fecha'])
# Quitando la hora de la fecha
dolar['Fecha'] = dolar['Fecha'].dt.date
# Poniendo la fecha como str
dolar['Fecha_Str'] = dolar['Fecha'].astype(str)

# Mergeando los datos con el precio del dolar
data = data.merge(dolar.drop(columns=['Fecha']), on='Fecha_Str')

# --------------------------------------------------------------------------------------------------------------------

# Multiplicando el precio por el cambio de moneda
data['Mínimo'] = np.round(data['Mínimo'] * data['Valor'], decimals=4)
data['Máximo'] = np.round(data['Máximo'] * data['Valor'], decimals=4)
data['Promedio'] = np.round(data['Promedio'] * data['Valor'], decimals=4)
# Eliminar la columna de valor
data.drop(columns=['Valor'], inplace=True)

# --------------------------------------------------------------------------------------------------------------------

# Sacando una columna con el año
data['Year'] = data['Fecha'].dt.year

# Dataset con la inflación
inflacion = pd.read_excel('./Data/Inflacion.xlsx')
data = data.merge(inflacion, on='Year')

# --------------------------------------------------------------------------------------------------------------------

# Agregando el porcentaje de desempleo
desempleo = pd.read_excel('./Data/Desempleo.xlsx')
data = data.merge(desempleo, on='Year')

# --------------------------------------------------------------------------------------------------------------------

# Poniendo la fecha como indice
data.set_index('Fecha', inplace=True)

# Ordenando los datos por fecha y por producto
data.sort_values(by=['Fecha', 'Producto'], inplace=True)

# --------------------------------------------------------------------------------------------------------------------

# Ordenando las columnas
data = data[['Producto', 'Clase', 'Mínimo', 'Máximo', 'Promedio', 'Inflacion', 'Precio_Dolar', 'Desempleo']]

data.head()

## <font color='maroon'> EDA </font>

In [ ]:
# Conocer las variables

# Tipo de cada variable
print(f'\n+ Tipo de datos por columna: \n{data.dtypes}')

# Conteo de valores nulos
print(f'\n+ Cantidad de nulos por columna: \n{data.isnull().sum()}')

# Valores únicos
print(f'\n+ Valores únicos por columna: \n{data.nunique()}')

In [ ]:
# Datos estadísticos de las variables
data.describe()

In [ ]:
# Mostrando fechas de inicio y fin de los datos, puras fechas sin hora
print('Fecha de inicio: ', data.index.min().date())
print('Fecha de fin: ', data.index.max().date())

In [ ]:
# Productos
tipos_de_productos = data['Producto'].unique()
print(f'Cantidad de Productos: {len(tipos_de_productos)}')

In [ ]:
# Porcentaje que representa cada producto
porcentaje = data['Producto'].value_counts(normalize=True).reset_index().sort_values(by='Producto', ascending=False)
# Multiplicando por 100 para obtener el porcentaje
porcentaje['Producto'] = np.round(porcentaje['Producto'] * 100, decimals=3)
# Haciendo una suma acumulativa
porcentaje['Cum_Sum'] = porcentaje['Producto'].cumsum()
porcentaje

___
## <font color='maroon'> Modelos Lineales </font>

In [ ]:
# Poniendo un seed para que siempre se obtenga el mismo resultado
np.random.seed(21)

# Random choice para seleccionar un producto al azar
producto = np.random.choice(tipos_de_productos)

# Graficando el promedio del producto seleccionado
data[data['Producto'] == producto]['Promedio'].plot(figsize=(15, 5), title=f'Precio promedio del producto {producto} en Nepal.')
plt.show()

In [ ]:
# Guardar la serie de tiempo de interés
data_prod = pd.DataFrame(data.loc[data.Producto == producto]['Promedio'])

In [ ]:
# Ver si los meses están completos
print(f'Primera fecha: {data_prod.index[0]}\nÚltima fecha: {data_prod.index[-1]}')

In [ ]:
data_lin = data_prod.drop(['2013-06-16', '2013-06-17', '2013-06-18', '2013-06-19', '2013-06-20', '2013-06-21',
                            '2013-06-25', '2013-06-26', '2013-06-27', '2013-06-28', '2013-06-30', '2021-05-01',
                            '2021-05-02', '2021-05-03', '2021-05-04', '2021-05-05', '2021-05-06', '2021-05-07',
                            '2021-05-08', '2021-05-09', '2021-05-10', '2021-05-11'])

In [ ]:
# Obtener un promedio mensual de los precios en lugar de mantener datos diarios
data_lin = data_lin.resample('M').mean()
data_lin

In [ ]:
# Dividir en datos de entrenamiento (80%) y prueba (20%)
div = int(data_lin.shape[0]*.8)
train = data_lin[:div+1]
test = data_lin[div:]

# Visualizar partición entrenamiento - prueba
fig, ax = plt.subplots(figsize=(10, 5))
train.plot(figsize=(8,5), ax=ax,)
test.plot(figsize=(8,5), ax=ax)
ax.legend(labels = ['train', 'test'])
plt.show()

In [ ]:
# Visualizar la distribución de la serie de tiempo
fig, ax = plt.subplots(3, 1, figsize=(8, 8))
sns.boxplot(x=train['Promedio'], ax=ax[0])
sns.histplot(x=train['Promedio'], ax=ax[1])
sns.lineplot(data=train['Promedio'], ax=ax[2])
ax[0].set(title='Boxplot', xlabel=None)
ax[1].set(title='Histograma', xlabel=None)
ax[2].set(title='Diagrama de línea', xlabel=None)
plt.tight_layout()

Dado que la serie no tiene una distribución normal, se transformará.

In [ ]:
def transform_series(train_data, method, plot=True):
    data_transformations = train_data.copy()
    # Escalamientos
    if method == 'min_max':
        min_max = MinMaxScaler()
        data_transformations['min_max'] = min_max.fit_transform(data_transformations.values.reshape(-1, 1))
    elif method == 'standard':
        scaler = StandardScaler()
        data_transformations['standar'] = scaler.fit_transform(data_transformations.values.reshape(-1, 1))
    elif method == 'log':
        data_transformations['log'] = np.log(data_transformations)
    elif method == 'box_cox':
        data_transformations['box_cox'] = power_transform(data_transformations.values.reshape(-1, 1), method='box-cox')
    else:
        raise ValueError('Método de transformación no válido.')
    print(f'Transformación {method} completada.')
    if plot:
        fig, ax = plt.subplots(3, 1, figsize=(8, 8))
        sns.boxplot(x=data_transformations[method], ax=ax[0])
        sns.histplot(x=data_transformations[method], ax=ax[1])
        sns.lineplot(data=data_transformations[method], ax=ax[2])
        ax[0].set(title="Boxplot", xlabel=None)
        ax[1].set(title="Histograma", xlabel=None)
        ax[2].set(title="Diagrama de línea", xlabel=None)
        plt.xticks(rotation=90)
        plt.tight_layout()
    return data_transformations

In [ ]:
train = transform_series(train, 'log')

In [ ]:
resultados = seasonal_decompose(train['log'], model='additive', period=12)

# Gráfica
fig, ax = plt.subplots(4, sharex=True, figsize=(8, 8))
resultados.observed.plot(ax=ax[0])   # Datos originales
ax[0].set_ylabel('Observed')
resultados.trend.plot(ax=ax[1])      # Tendencia
ax[1].set_ylabel('Trend')
resultados.seasonal.plot(ax=ax[2])   # Estacionalidad
ax[2].set_ylabel('Seasonal')
resultados.resid.plot(ax=ax[3])      # Residuos
ax[3].set_ylabel('Residual')
fig.tight_layout()

La serie de tiempo muestra una clara estacionalidad anual (cada 12 meses), así como una ligera tendencia a la alcista que era de esperarse por la inflación.

In [ ]:
# Función para graficar autocorrelación
def plot_acf_pacf(data, kwargs=dict()):
    f = plt.figure(figsize=(8,5))
    ax1 = f.add_subplot(121)
    plot_acf(data, zero=False, ax=ax1, **kwargs)
    ax2 = f.add_subplot(122)
    plot_pacf(data, zero=False, ax=ax2, method='ols', **kwargs)
    plt.show()
    
# Función para realizar la prueba de Dikcey-Fuller
def adf_test(timeseries):
    print("Results of Dickey-Fuller Test:")
    dftest = adfuller(timeseries, autolag="AIC")
    dfoutput = pd.Series(
        dftest[0:4],
        index=[
            "Test Statistic",
            "p-value",
            "#Lags Used",
            "Number of Observations Used",
        ],
    )
    for key, value in dftest[4].items():
        dfoutput["Critical Value (%s)" % key] = value
    print(dfoutput)
    
    if (dftest[1] <= 0.05) & (dftest[4]['5%'] > dftest[0]):
        print("\u001b[32mStationary\u001b[0m")
    else:
        print("\x1b[31mNon-stationary\x1b[0m")

In [ ]:
# Funciones de autocorrelación
plot_acf_pacf(train['log'], {'lags':25})

In [ ]:
# Prueba de Dikcey-Fuller para ver si la serie es estacionaria
adf_test(train['log'])

Como la serie no es estacionaria, se probarán diferenciaciones de primer y segundo orden.

In [ ]:
# Funciones de autocorrelación con diferenciación
diff1 = train['log'].diff().dropna()
print('ADF para derivada primer orden\n')
adf_test(diff1)

diff2 = train['log'].diff().diff().dropna()
print('ADF para derivada segundo orden\n')
adf_test(diff2)

In [ ]:
# Funciones de autocorrelación con diferenciación
plot_acf_pacf(diff1, {'lags':25})

In [ ]:
# Funciones de autocorrelación con diferenciación de segundo orden
plot_acf_pacf(diff2, {'lags':25})

In [ ]:
# Parte estacional
plot_acf_pacf(resultados.seasonal, {'lags':25})

In [ ]:
# Parte estacional con diferenciación
plot_acf_pacf(resultados.seasonal.diff().dropna(), {'lags':25})

In [ ]:
# Parte estacional con diferenciación
plot_acf_pacf(resultados.seasonal.diff().diff().dropna(), {'lags':25})

**Propuestas de modelos:**
Después de analizar los datos, se considera que podrían funcionar los siguientes modelos:
+ $ARIMA(1, 0, 3)$ porque el primer *lag* es significativo en la PACF y los primeros tres *lags* son significativos en la ACF
+ $ARIMA(2, 2, 2)$ porque los dos primeros *lags* son sifnigicantes en ambas gráficas (PACF y ACF), además de la diferenciación de segundo orden
+ $ARIMA(2, 1, 2)$ porque los dos primeros *lags* son sifnigicantes en ambas gráficas (PACF y ACF), además de la diferenciación

Siguiendo esta lógica, también se probarán las siguientes combinaciones:
+ $ARIMA(1, 0, 2)$
+ $ARIMA(1, 0, 1)$
+ $ARIMA(1, 2, 2)$
+ $ARIMA(2, 2, 1)$
+ $ARIMA(1, 2, 1)$
+ $ARIMA(1, 1, 2)$
+ $ARIMA(2, 1, 1)$
+ $ARIMA(1, 1, 1)$

Además se verá qué modelo sugiere el método `autoarima`:

In [ ]:
# Crear modelo
auto_model = auto_arima(train['log'],
                        start_p=1,
                        start_q=1,
                        test='adf',                   # para encontrar el 'd' óptimo
                        information_criterion='aic',  # se buscará reducir el AIC
                        m=1,             
                        d=1,          
                        seasonal=True,   
                        start_P=0, 
                        D=None, 
                        trace=True,
                        error_action='ignore',  
                        suppress_warnings=True, 
                        stepwise=True)

**Modelado**

In [ ]:
# Modelo 1
arima_model1 = ARIMA(train['log'], order=(1, 0, 3))
model1 = arima_model1.fit()
print(model1.summary())

In [ ]:
# Modelo 2
arima_model2 = ARIMA(train['log'], order=(2, 2, 2))
model2 = arima_model2.fit()
print(model2.summary())

In [ ]:
# Modelo 3
arima_model3 = ARIMA(train['log'], order=(2, 1, 2))
model3 = arima_model3.fit()
print(model3.summary())

In [ ]:
# Modelo 4
arima_model4 = ARIMA(train['log'], order=(1, 0, 2))
model4 = arima_model4.fit()
print(model4.summary())

In [ ]:
# Modelo 5
arima_model5 = ARIMA(train['log'], order=(1, 0, 1))
model5 = arima_model5.fit()
print(model5.summary())

In [ ]:
# Modelo 6
arima_model6 = ARIMA(train['log'], order=(1, 2, 2))
model6 = arima_model6.fit()
print(model6.summary())

In [ ]:
# Modelo 7
arima_model7 = ARIMA(train['log'], order=(2, 2, 1))
model7 = arima_model7.fit()
print(model7.summary())

In [ ]:
# Modelo 8
arima_model8 = ARIMA(train['log'], order=(1, 2, 1))
model8 = arima_model8.fit()
print(model8.summary())

In [ ]:
# Modelo 9
arima_model9 = ARIMA(train['log'], order=(1, 1, 2))
model9 = arima_model9.fit()
print(model9.summary())

In [ ]:
# Modelo 10
arima_model10 = ARIMA(train['log'], order=(2, 1, 1))
model10 = arima_model10.fit()
print(model10.summary())

In [ ]:
# Modelo 11
arima_model11 = ARIMA(train['log'], order=(1, 1, 1))
model11 = arima_model11.fit()
print(model11.summary())

In [ ]:
# Modelo 12
model12 = SARIMAX(train['log'], order=(1, 1, 1), seasonal_order=(2, 0, 1, 12))
sarima1 = model12.fit()
sarima1.summary()

In [ ]:
# Modelo 13
model13 = SARIMAX(train['log'], order=(0, 1, 0), seasonal_order=(1, 0, 1, 12))
sarima2 = model13.fit()
sarima2.summary()

**Ahora, se analizarán los resultados**

In [ ]:
# Función para separar los valores del resumen del modelo
def extract_values(text):
    values = {}
    lines = text.split("\n")
    for line in lines:
        line = line.strip()
        if line:
            words = line.split()
            if len(words) >= 2:
                try:
                    value = float(words[-1])
                    key = " ".join(words[:-1])
                    values[key] = value
                except ValueError:
                    pass
    return values

In [ ]:
# Crear una lista con todos los resultados de los modelos
results = []
for i in [model1, model2, model3, model4, model5, model6, model7, model8, model9, model10, model11, sarima1, sarima2]:
    results.append(extract_values(i.summary().as_text()))

In [ ]:
# Convertir a DataFrame
r = pd.DataFrame(results)


# Visualizar
br1 = np.arange(len(list(r.iloc[:, 0])))
br2 = [x + 0.25 for x in br1]
plt.bar(br1, r.iloc[:, 2], color ='teal', width = 0.25, label ='AIC')
plt.bar(br2, r.iloc[:, 3], color ='cornflowerblue', width = 0.25, label ='BIC')
plt.title('Complejidad y desempeño de modelos')
plt.xlabel('Modelo')
plt.ylabel('Valor')
plt.xticks([r + 0.25 for r in range(13)], ['arima_1', 'arima_2', 'arima_3', 'arima_4', 'arima_5', 'arima_6', 'arima_7',
                                           'arima_8', 'arima_9', 'arima_10', 'arima_11', 'sarima_1', 'sarima_2'],
           rotation=45)
plt.legend()
plt.show()

**_`Observaciones:`_** <br>
+ **Log-Likelihood**: conforme el valor de la verosimilitud sea mayor, es mejor. En este caso, todos los modelos estuvieron entre $-3$ y $-12$. El mejor fue el model 12 (primer SARIMAX). 

+ **Criterio de información de Akaike**: si este valor es menor, es mejor. Por lo podemos *rankear* a los modelos de la siguiente manera:
1. Modelo 9 ($18.103$)
1. Modelo 5 ($18.128$)
1. Modelo 12 ($18.482$)
1. Modelo 4 ($18.977$)
1. Modelo 13 ($18.987$)
1. Modelo 3 ($19.784$)
1. Modelo 1 ($20.950$)
1. Modelo 10 ($22.587$)
1. Modelo 11 ($23.825$)
1. Modelo 7 ($27.898$)
1. Modelo 2 ($29.803$)
1. Modelo 6 ($30.594$)

+ **Criterio de información bayesiano**: si este valor es menor, es mejor. Por lo podemos *rankear* a los modelos de la siguiente manera:
1. Modelo 12 ($24.034$)
1. Modelo 13 ($25.940$)
1. Modelo 11 ($30.778$)
1. Modelo 9 ($27.372$)
1. Modelo 5 ($27.451$)
1. Modelo 4 ($30.631$)
1. Modelo 3 ($31.372$)
1. Modelo 10 ($31.857$)
1. Modelo 1 ($34.935$)
1. Modelo 7 ($37.114$)
1. Modelo 8 ($38.262$)
1. Modelo 6 ($39.811$)
1. Modelo 2 ($41.324$)

+ **Ljung-Box**: en todos los casos el *p-value* es mayor a $0.05$, así que no se rechaza la hipótesis nula y los datos se distribuyen de forma independiente
+ **Heterocedasticidad**: en la mayoría de los casos el *p-value* es mayor a $0.05$ y no se rechaza la hipótesis nula, por lo que los residuos muestran varianza cambiante a excepción del modelo 3 y 7
+ **Jarque-Bera**: todos los modelos tienen un *p-value* es mayor a $0.05$ en esta prueba, por lo que no se rechaza la hipótesis nula y los datos se distribuyen normalmente

El mejor modelo parece ser el 12 ($SARIMA(1, 1, 1) \times (2, 0, 1, 12)$)

In [ ]:
sarima1.plot_diagnostics(figsize=(10, 10))
plt.gcf().autofmt_xdate()
plt.show()

**`Observaciones:`**
+ Se pueden ver los residuales sin tendencia ni estacionalidad
+ En el histograma se puede ver que los residuales no son muy distintos a una distribución normal, lo cual es muy bueno
+ En la gráfica `Normal Q-Q` podemos ver que los residuales tienen un comportamiento similar a la línea de referencia
+ Los residuales no tienen correlaciones

In [ ]:
# Predicción
# Pronóstico
y_h = np.exp(sarima1.predict(start=train.shape[0], end=train.shape[0]+test.shape[0], dynamic=False)).to_frame()
fig, ax = plt.subplots(figsize = (10, 5))
train['Promedio'].plot(ax = ax)
test.plot(ax = ax)
y_h.plot(ax = ax)
ax.legend(labels = ['train', 'test', 'forecast'])
plt.show()

___
## <font color='maroon'> Parte Univariada con Deep Learning </font>

In [ ]:
# Para la parte univariada solo con la variable del promedio.
df = data_prod[["Promedio"]]
obj = NN_maker(data=df, n_steps= 7, horizont=14)

In [ ]:
obj.plot_serie()

Al estar tan sesgada la serie, vamos a aplicar un logaritmo, a ver si mejora, ya que tiene una cola positiva, el logaritmo puede tender a mejorar la distribución de los datos.

In [ ]:
obj.transform_data()
obj.plot_serie()

En efecto ahora hay una distribución mucho más normal, vamos a proceder con el modelado de los datos, primero vamos a hacer un modelo muy simple mlp.

In [ ]:
# Separamos los datos en train, val y test
X_train, X_val, X_test, y_train, y_val, y_test= obj.train_val_test_split()

# Imprimir dimensiones de los datos de entrenamiento, validación y test
print('Datos de entrenamiento', X_train.shape, y_train.shape)
print('Datos de validación', X_val.shape, y_val.shape)
print('Datos de test', X_test.shape, y_test.shape)

In [ ]:
# Ejecutamos un modelo sencillo con solo 1 capa oculta y 32 neuronas
num_hidden_layers = 1
num_neurons = 32
dropout_rate = None

model_mlp_1 = obj.MLP_builder(num_hidden_layers=num_hidden_layers, X_train=X_train, X_val=X_val, X_test=X_test, y_train=y_train, 
                        y_val=y_val, y_test=y_test, num_neurons=num_neurons, log=True, plot=True)

In [ ]:
# Ahora corremos con 2 capas ocultas y 64 neuronas, agregamos dropout para evitar overfitting
num_hidden_layers = 2
num_neurons = 64
dropout_rate = 0.2

model_mlp_2 = obj.MLP_builder(num_hidden_layers=num_hidden_layers, X_train=X_train, X_val=X_val, X_test=X_test, y_train=y_train, 
                        y_val=y_val, y_test=y_test, num_neurons=num_neurons, log=True, plot=True, dropout_rate=dropout_rate)

Parece que las capas ocultas evitaron que siga aprendiendo, voy a intentar, dejando el dropout y quitando una oculta.

In [ ]:
# Vamos a quitar una capa oculta y jugar con el dropout.
num_hidden_layers = 1
num_neurons = 64
dropout_rate = 0.2

model_mlp_3 = obj.MLP_builder(num_hidden_layers=num_hidden_layers, X_train=X_train, X_val=X_val, X_test=X_test, y_train=y_train, 
                        y_val=y_val, y_test=y_test, num_neurons=num_neurons, log=True, plot=True, dropout_rate=dropout_rate)

### <font color='teal'> CNN </font>

In [ ]:
# Separamos en train, val y test. y reordenamos en forma tensorial con el parametro conv.
X_train, X_val, X_test, y_train, y_val, y_test = obj.train_val_test_split(conv=True)

# Imprimir dimensiones de los datos de entrenamiento, validación y test
print('Datos de entrenamiento', X_train.shape, y_train.shape)
print('Datos de validación', X_val.shape, y_val.shape)
print('Datos de test', X_test.shape, y_test.shape)

In [ ]:
# Ejecutamos nuestra primer red, solo una capa oculta
num_hidden_layers = 1
num_neurons = 256
num_filters = 64
kernel_size = 2

model_cnn_1 = obj.cnn_builder(num_hidden_layers=num_hidden_layers, num_neurons=num_neurons,
                                    num_filters=num_filters, kernel_size=kernel_size, pool_size=2,
                                    X_train=X_train, X_val=X_val, X_test=X_test, y_train=y_train,
                                    y_val=y_val, y_test=y_test, plot=True, log=True)

In [ ]:
# Dió muy buen resultado, pero los picos todavía no me convencen
num_hidden_layers = 1
num_neurons = 256
num_filters = 64
kernel_size = 2

model_cnn_2 = obj.cnn_builder(num_hidden_layers=num_hidden_layers, num_neurons=num_neurons,
                                    num_filters=num_filters, kernel_size=kernel_size, pool_size=2,
                                    X_train=X_train, X_val=X_val, X_test=X_test, y_train=y_train,
                                    y_val=y_val, y_test=y_test, plot=True, log=True)

In [ ]:
# veamos si el dropout hace algun buen efecto en la red
num_hidden_layers = 1
num_neurons = 256
num_filters = 64
kernel_size = 2
dropout_rate = 0.5

model_cnn_3 = obj.cnn_builder(num_hidden_layers=num_hidden_layers, num_neurons=num_neurons,
                                    num_filters=num_filters, kernel_size=kernel_size, pool_size=2,
                                    X_train=X_train, X_val=X_val, X_test=X_test, y_train=y_train,
                                    y_val=y_val, y_test=y_test, dropout_rate=dropout_rate, plot=True, log=True)

### <font color='teal'>LSTM </font>

In [ ]:
units_lstm = 50
capas_ocultas_dense = 1
units_dense = 32

model_lstm_1 = obj.lstm_builder(units_lstm=units_lstm, capas_ocultas_lstm=1, capas_ocultas_dense=capas_ocultas_dense, 
                                     units_dense=units_dense, X_train = X_train, X_val=X_val, X_test=X_test, y_train=y_train, 
                                     y_val=y_val, y_test=y_test, plot=True, dropout=None, log=True)

Modelo muy malo, vamos a ver si con otros parámetros puede mejorar.

In [ ]:
# Pesimo modelo, vamos agregando más parámetros
units_lstm = 50
capas_ocultas_dense = 3
units_dense = 128

model_lstm_2 = obj.lstm_builder(units_lstm=units_lstm, capas_ocultas_lstm=1, capas_ocultas_dense=capas_ocultas_dense, 
                                     units_dense=units_dense, X_train = X_train, X_val=X_val, X_test=X_test, y_train=y_train, y_val=y_val, 
                                     y_test=y_test, plot=True, dropout=None, log=True)

In [ ]:
# Vamos a ver si con más unidades lstm se puede mejorar el modelo
units_lstm = 500
capas_ocultas_dense = 5
units_dense = 526

model_lstm_3 = obj.lstm_builder(units_lstm=units_lstm, capas_ocultas_lstm=1, capas_ocultas_dense=capas_ocultas_dense, 
                                     units_dense=units_dense, X_train = X_train, X_val=X_val, X_test=X_test, y_train=y_train, y_val=y_val, 
                                     y_test=y_test, plot=True, dropout=None, log=True)

### <font color='teal'> CNN-LSTM </font>

In [ ]:
# Separamos en train, val y test. y reordenamos en forma tensorial con el parametro conv.
X_train, X_val, X_test, y_train, y_val, y_test = obj.train_val_test_split(conv=True)

n_features = 1
n_seq = 1
n_steps = 7
X_train = X_train.reshape((X_train.shape[0], n_seq, n_steps, n_features))
X_val = X_val.reshape((X_val.shape[0], n_seq, n_steps, n_features))
X_test = X_test.reshape((X_test.shape[0], n_seq, n_steps, n_features))

print('Datos de entrenamiento', X_train.shape, y_train.shape)
print('Datos de validación', X_val.shape, y_val.shape)
print('Datos de test', X_test.shape, y_test.shape)

In [ ]:
# Empezamos 
blocks = 1
filters = 10
units_lstm = 30

model_cnn_lstm_1 = obj.cnn_lstm_builder(blocks=blocks, filters=filters, units_lstm=units_lstm,  X_train=X_train, X_val=X_val, 
                                             X_test=X_test, y_train=y_train, y_val=y_val, y_test=y_test, plot=True, 
                                             dropout=None, log=True)

In [ ]:
blocks = 1
filters = 40
units_lstm = 50

model_cnn_lstm_2 = obj.cnn_lstm_builder(blocks=blocks, filters=filters, units_lstm=units_lstm,  X_train=X_train, X_val=X_val, 
                                             X_test=X_test, y_train=y_train, y_val=y_val, y_test=y_test, plot=True, 
                                             dropout=None, log=True)

In [ ]:
blocks = 1
filters = 256
dropout_rate = 0.2
units_lstm = 100

model_cnn_lstm_3 = obj.cnn_lstm_builder(blocks=blocks, filters=filters, units_lstm=units_lstm,  X_train=X_train, X_val=X_val, 
                                             X_test=X_test, y_train=y_train, y_val=y_val, y_test=y_test, plot=True, 
                                             dropout=dropout_rate, log=True)

El mejor modelo conseguido fue una CNN, vamos a hacer un optuna con esa arquitectura.

In [ ]:
from tensorflow.keras.layers import Dense, MaxPooling1D, Dropout, Conv1D, Flatten
from tensorflow.keras.models import Sequential
from tensorflow import keras
import optuna

# Separamos los datos en train, val y test
X_train, X_val, X_test, y_train, y_val, y_test= obj.train_val_test_split(conv=True)


def objective(trial):
    # Definir los hiperparámetros que queremos optimizar
    filters = trial.suggest_int("filters", 32, 128)
    dropout = trial.suggest_float("dropout", 0.2, 0.5)
    dense_size = trial.suggest_int("dense_size", 64, 256)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)

    # Construir el modelo con los hiperparámetros elegidos
    model = Sequential()
    model.add(Conv1D(filters=filters, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(dropout))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(dense_size, activation='relu'))
    model.add(Dense(1))

    # Compilar el modelo con el optimizador y la función de pérdida
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse')

    # Entrenar el modelo y obtener las predicciones
    history = model.fit(X_train, y_train,
            validation_data=(X_val, y_val),
            batch_size=256,
            epochs=200,
            verbose=False)
    val_loss = history.history['val_loss'][-1]
    return val_loss

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Imprimir los resultados
print('Best trial:', study.best_trial.params)
print('Best validation loss:', study.best_trial.value)

In [ ]:
# Imprimimos los mejores parámetros
dict_params = study.best_params
dict_params

In [ ]:
# Ejecutamos nuestra red con datos optimizados
dense_size = dict_params["dense_size"]
dropout = dict_params["dropout"]
filters = dict_params["filters"]
lr = dict_params["learning_rate"]

# Construir el modelo con los hiperparámetros elegidos
model = Sequential()
model.add(Conv1D(filters=filters, kernel_size=2, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(dropout))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(dense_size, activation='relu'))
model.add(Dense(1))

# Compilar el modelo con el optimizador y la función de pérdida
optimizer = keras.optimizers.Adam(learning_rate=lr)
model.compile(optimizer=optimizer, loss='mse')

obj.train_models(model=model, X_train=X_train, X_val=X_val, X_test=X_test, 
                 y_train=y_train, y_val=y_val, y_test=y_test, log=True, plot=True)

Pues salió bastante parecido el resultado del modelo solo que con menos neuronas, filtros, etc. Entonces se puede considerar que nos funcionó el optuna.

___
## <font color='maroon'> Parte Multivariada con Deep Learning </font>
### Creación del dataset
- Al tener datos que contiene las variables de Mínimo, Inflación, Precio del Dolar y Porcentaje de Desempelo para predecir el precio promedio del producto tenemos que crear el dataset de manera que las variables predictoras sean del día anterior al que se quiere predecir.
    - Es decir, si queremos predecir el precio promedio del producto para el día 2021-06-01, las variables predictoras serán las del día 2021-05-31.
- Se decidió hacer una ventana de 7, al tener datos diarios hacemos que nuestras ventanas de tiempo sean semanales y podamos realizar un mejor uso de los datos.
- Las variables:
    - Inflación y Porcentaje de Desempleo: parecieran que son estaticas, sin embargo al tener temporalidad más grande no afectan mucho a la predicción.
    - Mínimo y Precio del Dolar: son las variables que más afectan a la predicción, ya que son las que más varían en el tiempo y afectan directamente al precio promedio del producto.

In [ ]:
# Filtramos el dataset para quedarnos solo con el producto que queremos predecir
data_prod_multi = data[data['Producto'] == producto]
# Eliminando fechas
data_prod_multi = data_prod_multi.drop(['2013-06-16', '2013-06-17', '2013-06-18', '2013-06-19', '2013-06-20', '2013-06-21',
                                        '2013-06-25', '2013-06-26', '2013-06-27', '2013-06-28', '2013-06-30', '2021-05-01',
                                        '2021-05-02', '2021-05-03', '2021-05-04', '2021-05-05', '2021-05-06', '2021-05-07',
                                        '2021-05-08', '2021-05-09', '2021-05-10', '2021-05-11'])

# Tomamos las 3 variables que se usarán como predictores y la de respuesta
data_multi = data_prod_multi[['Mínimo', 'Inflacion', 'Precio_Dolar', 'Desempleo', 'Promedio']]
# Desplazamos las variables predictoras 1 día
data_multi[['Mínimo', 'Inflacion', 'Precio_Dolar', 'Desempleo']] = data_multi[['Mínimo', 'Inflacion', 'Precio_Dolar', 'Desempleo']].shift(1)
# Quitando nulos
data_multi = data_multi.dropna()
# Convertimos el dataframe a un array
data_multi = data_multi.values
# Mostrando los tamaños
print(f'Forma del array: \n- {data_multi.shape}\n')
print(f'Primeros 5 elementos del array: \n{data_multi[:5]}')

In [ ]:
# Dividiendo en X e y, además de tomar un n-steps de 7 para que sea semanal
X, y = split_multivariate_sequence(data_multi, 7)

In [ ]:
# Dividiendo en train y test
X_train, X_test, y_train, y_test = train_test_split_multi(X=X, 
                                                          y=y,
                                                          train=0.8)

### <font color='teal'> MLP </font>

In [ ]:
# Creando el modelo de MLP con 6 capas ocultas y 64 neuronas por capa
model_mlp_multi, history_mlp_mlti = gen_MLP_model(X=X_train, y=y_train, val_split=0.2, input_shape=(X_train.shape[1], X_train.shape[2]),
                                                  activation='relu', num_layers=6, num_neurons=64,
                                                  optimizer='Adagrad', lr=0.01, loss='mse', metrics=['mae'],
                                                  patience=25, epochs=500, verbose=0,
                                                  X_test=X_test, y_test=y_test, index=np.arange(0, 464),
                                                  plot_history=True
                                                  )

### <font color='teal'> CNN </font>

In [ ]:
# Creando el modelo de CNN con 5 capas de convolución con 128 filtros y 5 capas densas con 100 neuronas
model_cnn_multi, history_cnn_multi = gen_CNN_model(X=X_train, y=y_train, val_split=0.2, input_shape=(X_train.shape[1], X_train.shape[2]),
                                                    num_layers_cnn=5, num_filters=128, kernel_size=4, padding='same',
                                                    activation='relu', num_layers_dense=5, num_neurons=100,
                                                    optimizer='Adagrad', lr=0.01, loss='mse', metrics=['mae'],
                                                    patience=25, epochs=500, verbose=0,
                                                    X_test=X_test, y_test=y_test, index=np.arange(0, 464),
                                                    plot_history=True
                                                    )

### <font color='teal'> LSTM </font>

In [ ]:
# Creando el modelo de LSTM con 5 capas de LSTM con 100 unidades y 5 capas densas con 100 neuronas
model_lstm_multi, history_lstm_multi = gen_LSTM_model(X=X_train, y=y_train, val_split=0.2, input_shape=(X_train.shape[1], X_train.shape[2]),
                                                      num_layers_lstm=5, activation_lstm='relu', num_units_lstm=100, bidireccional=True,
                                                      activation='relu', num_layers_dense=5, num_neurons=100,
                                                      optimizer='Adagrad', lr=0.01, loss='mse', metrics=['mae'],
                                                      patience=25, epochs=500, verbose=0,
                                                      X_test=X_test, y_test=y_test, index=np.arange(0, 464),
                                                      plot_history=True
                                                      )

### <font color='teal'> CNN-LSTM </font>

In [ ]:
model_cnn_lstm_multi, history_cnn_lstm_multi = gen_CNN_LSTM_model(X=X_train, y=y_train, val_split=0.2, input_shape=(X_train.shape[1], X_train.shape[2]),
                                                                  num_layers_cnn=5, num_filters=128, kernel_size=4, padding='same',
                                                                  num_layers_lstm=5, activation_lstm='relu', num_units_lstm=100,
                                                                  activation='relu', num_layers_dense=5, num_neurons=100,
                                                                  optimizer='Adagrad', lr=0.01, loss='mse', metrics=['mae'],
                                                                  patience=0, epochs=500, verbose=0,
                                                                  X_test=X_test, y_test=y_test, index=np.arange(0, 464),
                                                                  plot_history=True
                                                                  )

### <font color='teal'> Conclusiones </font>
- Modelos encontrados:
    - El mejor modelo encontrado es el de CNN con 0.89 de r2.
    - El segundo mejor modelo encontrado es del MLP con 0.88 de r2.
- Selección del Modelo
    - Se selecciona el modelo de MLP, ya que aunque tiene un punto menos de r2, el tiempo de entrenamiento es mucho menor que el de CNN (aprox. 5.5 veces más)

___
## <font color='maroon'> Clasificación de Series </font>
### <font color='maroon'> Preparación Dataset </font>
- En este paso se realizará el procesamiento para lograr la clasificación de las series de tiempo

In [ ]:
# Ver las clases existentes
data['Clase'].value_counts().plot(kind='bar', figsize=(12, 8), color='teal')
plt.title('Clases', fontsize=15)
plt.ylabel('Cantidad de productos', fontsize=12)
plt.xlabel('Clase', fontsize=12)
plt.tight_layout()
plt.show()

# Conteo
print(f'Hay {data["Clase"].nunique()} clases de productos distintas.')

Debido a que son demasiadas clases y están imbalanceadas, se agruparán los productos en $3$ distintas clasificaciones:
+ Frutas
+ Verduras
+ Otros

In [ ]:
# Diccionario con las clases
dict_clases = {0: 'Others', 1: 'Fruits', 2: 'Veggies'}

# Obteniendo las nuevas categorías
data = get_categories(data)
# Mostrando
data.head()

In [ ]:
# Obteniendo las longitudes de las series de tiempo
products_ts = {}
for prod in data['Producto'].unique():
    products_ts[prod] = [np.array(data.loc[data.Producto==prod]['Promedio']), data.loc[data.Producto==prod]['y'][0]]

# Ver las diferentes longitudes de las series de tiempo
lens = []
for values in products_ts.values():
    lens.append(len(values[0]))

# Mostrando cantidad de longitudes diferentes que tenemos
print(f'Hay {len(set(lens))} longitudes diferentes de series de tiempo.')

In [ ]:
# Obteniendo la cantidad de productos por longitud de serie de tiempo de cada clase
plot_length_ts(products_ts)

- Al tener series con longitudes diferentes se realizará un padding para que todas las series tengan la misma longitud.
- Se decidió que la longitud de las series de tiempo sea de 1250 datos, ya que es el punto medio de las 3 distribuciones.
    - Para realizarlo se hará un padding repitiendo el patrón de las series que tengan menos de 1250 datos y se truncarán las series que tengan más de 1250 datos.

In [ ]:
# Obteniendo nuestro dataset transformado, con el umbral de 1250
new_data, labels = TransformData(products_ts=products_ts, umbral=1250).apply_transformation()

In [ ]:
# Mostrando 5 ejemplos del nuevo dataset
print(f'Datos X: \n{new_data[:5]}')
print(f'\nDatos y: \n{labels[:5]}')

In [ ]:
# Dividiendo en train y test
X_train, X_test, y_train, y_test = split_data_clasification(data=new_data, labels=labels, test_size=0.2, shuffle=True)

# Lista con las etiquetas
etiquetas =  [dict_clases.get(0), dict_clases.get(1), dict_clases.get(2)]

### <font color='maroon'> Clasificación con Machine Learning </font>
#### <font color='teal'> XGBoost </font>

In [ ]:
# Crear y ajustar el modelo XGBoost
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X_train, y_train)

# Realizar predicciones con todos los datos
y_pred = xgb_model.predict(new_data)

# Evaluar el modelo
get_evaluation('XGBoost', labels, y_pred, etiquetas) 

#### <font color='teal'> Naive Bayes </font>

In [ ]:
# Crear y ajustar el modelo Naive Bayes
nb_model = GaussianNB()
nb_model.fit(X_train, y_train)

# Realizar predicciones con todos los datos
y_pred = nb_model.predict(new_data)

# Evaluar el modelo
get_evaluation('Naive Bayes', labels, y_pred, etiquetas)

#### <font color='teal'> Logistic Regression </font>

In [ ]:
# Crear y ajustar el modelo de regresión logística
lr = LogisticRegression()
lr.fit(X_train, y_train)

# Realizar predicciones con todos los datos
y_pred = lr.predict(new_data)

# Evaluar el modelo
get_evaluation('Regresión Logística', labels, y_pred, etiquetas)

#### <font color='teal'> K-Means </font>

In [ ]:
X_train

In [ ]:
# Crea el objeto KMeans
kmeans = KMeans(n_clusters=4)  # Ponemos 4 clústeres, porque sabemos que hay 4 etiquetas
# Ajusta el modelo con tus datos de entrenamiento
kmeans.fit(X_train)

# Predice los clústeres de tus datos de prueba
y_pred = kmeans.predict(X_test)

# Evaluar el modelo
get_evaluation_kmeans('KMeans', X_test, y_pred)

#### <font color='teal'> Conclusiones </font>
- Al tener tan pocos datos el entrenamiento de los modelos es muy rápido, por lo que esta no será una variable a considerar para la selección del modelo.
- El mejor modelo encontrado es el de XGBoost con 0.91 de accuracy.


### <font color='maroon'> Clasificación con Deep Learning </font>